# Vaccinatie ChatBot Huisartspraktijk

In [1]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [2]:
from dotenv import load_dotenv
import os

# Load .env variables and allow override if already set
load_dotenv(override=True)

# Load your keys
openai_api_key = os.getenv('OPENAI_API_KEY')
openrouter_api_key = os.getenv('OPENROUTER_API_KEY')  # this is used for Claude via OpenRouter
google_api_key = os.getenv('GOOGLE_API_KEY')

# Print summary for debugging
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if openrouter_api_key:
    print(f"OpenRouter API Key (for Claude) exists and begins {openrouter_api_key[:8]}")
else:
    print("OpenRouter API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")


OpenAI API Key exists and begins sk-proj-
OpenRouter API Key (for Claude) exists and begins sk-or-v1
Google API Key exists and begins AIzaSyDA


In [3]:
# Initialize

openai = OpenAI()
MODEL = 'gpt-4o-mini'

In [4]:
# === Multilingual system messages ===
system_messages = {
    "NL": (
        "Je bent een vriendelijke en informatieve assistent die alleen vragen beantwoordt over de griepprik en de pneumokokkenprik. "
        "Je geeft duidelijke en begrijpelijke informatie over deze vaccinaties, zoals wie in aanmerking komt, wanneer de vaccinaties plaatsvinden, "
        "mogelijke bijwerkingen en waarom de vaccinaties belangrijk zijn. "
        "Als iemand een vraag stelt die niets met de griepprik of pneumokokkenprik te maken heeft, geef je geen antwoord en verwijs je de patiënt vriendelijk "
        "door naar de huisarts of assistente. Gebruik een rustige en behulpzame toon. "
        "Je doel is om patiënten goed te informeren zodat ze een weloverwogen keuze kunnen maken."
    ),
    "EN": (
        "You are a friendly and informative assistant who only answers questions about the flu shot and the pneumococcal vaccine. "
        "You provide clear, understandable information about these vaccines, such as who qualifies, when they are given, possible side effects, and why they are important. "
        "If someone asks about a different topic, politely inform them that you can only assist with questions about these two vaccines, and recommend they speak to their doctor or nurse."
    )
}

In [5]:
# --- Chat function ---
def chat(message, history, language):
    full_messages = [{"role": "system", "content": system_messages[language]}] + history + [{"role": "user", "content": message}]
    stream = openai.chat.completions.create(model=MODEL, messages=full_messages, stream=True)

    collected = ""
    for chunk in stream:
        if chunk.choices[0].delta.content:
            collected += chunk.choices[0].delta.content
            yield {"role": "assistant", "content": collected}

In [6]:
# --- Gradio UI ---
chat_ui = gr.ChatInterface(
    fn=chat,
    additional_inputs=[gr.Radio(["NL", "EN"], value="NL", label="Taal / Language")],
    title="💉 Vaccine Info Chatbot",
    description="Stel je vraag over de griepprik of pneumokokkenprik. Ask about the flu or pneumococcal vaccine.",
    theme="soft",
    type="messages"
)

chat_ui.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
